In [ ]:
import plotly.express as px
import open3d as o3d
import numpy as np

ModuleNotFoundError: No module named 'open3d'

In [ ]:
mesh_file = "/cow.ply"
mesh = o3d.io.read_triangle_mesh(mesh_file ) # Read the point cloud

vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)

pcloud_file = "/mnt/e/pcloud/_frames_3_to_3_noise.ply"
pcloud = o3d.io.read_point_cloud(pcloud_file)
points = np.asarray(pcloud.points)


In [ ]:
import plotly.graph_objects as go
import numpy as np

x,y,z = np.asarray(mesh.vertices).T
i,j,k = np.asarray(mesh.triangles).T

x_p,y_p,z_p = np.asarray(pcloud.points).T

fig = go.Figure(data=[
    go.Mesh3d(
       x=x, y=y, z=z,
       i=i, j=j, k=k,
       #alphahull=5,
       #opacity=0.4,
       color='red'),
    go.Scatter3d(x=x_p, y=y_p, z=z_p,mode="markers", marker=dict(color='blue', size=1)),
   ],
layout=dict(
   scene=dict(aspectmode="data")
))
fig = fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np

x,y,z = np.asarray(mesh.vertices).T
i,j,k = np.asarray(mesh.triangles).T

x_p,y_p,z_p = np.asarray(pcloud.points).T

fig = go.Figure(data=[
    go.Mesh3d(
       x=x, y=y, z=z,
       i=i, j=j, k=k,
       facecolor=np.random.rand(x.shape[0],3)*255,
       color='red'),
    go.Scatter3d(x=x_p, y=y_p, z=z_p,mode="markers", marker=dict(color='blue', size=1)),
   ],
layout=dict(
   scene=dict(aspectmode="data")
))
fig = fig.show()

In [ ]:
p0 = vertices[triangles][:, 0, :]
p1 = vertices[triangles][:, 1, :]
p2 = vertices[triangles][:, 2, :]

u = p1 - p0
v = p2 - p1
face_center = (p0 + p1 + p2) / 3
face_normal = np.cross(u, v)
surface_area = np.linalg.norm(face_normal, axis=-1)/2
face_normal = face_normal / (2*surface_area)[:, None]

In [ ]:
def _edges_pos_for_plot(edges_start, edges_end):
    edge_x = []
    edge_y = []
    edge_z = []
    for start, end in zip(edges_start, edges_end):
        x0, y0, z0 = start
        x1, y1, z1 = end
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_z.append(z0)
        edge_z.append(z1)
        edge_z.append(None)
    return edge_x, edge_y, edge_z

def plot_3dlines(edges_start, edges_end, **kwargs):
    edge_x, edge_y, edge_z = _edges_pos_for_plot(edges_start, edges_end)

    return go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        **kwargs)

In [ ]:
import plotly.io as pio

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            mode="markers",
            marker=dict(color="blue", size=4),
        ),
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            #opacity=0.4,
            # facecolor="blue"
        ),
        plot_3dlines(
            edges_start=face_center,
            edges_end=face_center + 0.05 * face_normal,
            line=dict(width=2, color="black"),
            hoverinfo="none",
            mode="lines",
        ),
    ],
    layout=dict(
        scene=dict(aspectmode="data"), scene_camera=dict(eye=dict(x=0.5, y=0, z=0.8))
    ),
)
fig.update_layout(
    scene=dict(aspectmode="data"),
    title=dict(text=f"Normals", x=0.5),
    autosize=True,
    height=1000,
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
)

fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig = fig.show()

In [ ]:
np.linalg.inv([[2,0,0],[0,1,0],[0,0,1]])

In [ ]:
transform_to_tri_cc = np.stack([np.eye(4) for _ in triangles])

def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

for i in range(len(transform_to_tri_cc)):
    rotate = np.eye(4)
    rotate[:3,0] = p1[i]-p0[i]
    rotate[:3,1] = p2[i]-p0[i]
    rotate[:3,2] = face_normal[i]

    translate = np.eye(4)
    rotate[:3,3] = p0[i]

    transform_to_tri_cc[i] = np.linalg.inv(rotate@translate)

#np.linalg.norm(transform_to_tri_cc[-1][:3,:3], axis=0)

#transform_to_tri_cc[:,:3,0] = p1-p0
#transform_to_tri_cc[:,:3,1] = p2-p0
#transform_to_tri_cc[:,:3,2] = face_normal[i]
#transform_to_tri_cc[:,:3,3] = p0[i]
#transform_to_tri_cc = np.linalg.inv(transform_to_tri_cc)

In [ ]:
points_h = np.hstack((points, np.ones((len(points), 1), )))
projected = (transform_to_tri_cc[0]@points_h.T).T

In [ ]:
print(points_h.shape)
print(transform_to_tri_cc.shape)

In [ ]:
np.einsum("ijj,lj->ilj", transform_to_tri_cc, points_h)

In [ ]:
projected_points_per_triangle = []
per_triangle_resolution = []
normal_eps_pos = 0.01
normal_eps_neg = 0.001
for i, transform in enumerate(transform_to_tri_cc):
    projected = (transform @ points_h.T).T
    is_within_triangle = projected[
        (projected[:, 0] >= 0)
        & (projected[:, 1] >= 0)
        & ((projected[:, 1] + projected[:, 0]) <= 1)
        & (
            ((projected[:, 2] > 0) & (projected[:, 2] <= normal_eps_pos))
            | ((projected[:, 2] < 0) & (projected[:, 2] >= normal_eps_neg))
        )
    ]
    if len(is_within_triangle) > 0:
        projected_points_per_triangle.append(is_within_triangle)
        per_triangle_resolution.append(len(is_within_triangle))# / surface_area[i])
    else:
        projected_points_per_triangle.append([])
        per_triangle_resolution.append(0)

per_triangle_resolution = np.stack(per_triangle_resolution)

In [ ]:
print(transform_to_tri_cc.shape)
print(per_triangle_resolution.shape)

In [ ]:
colorscale_0 = np.array([0x44, 0x01, 0x54, 1], dtype=float)
colorscale_1 = np.array([0xfd, 0xe7, 0x25, 1], dtype=float)

def multi_add(*args):
    if len(args)>1:
        return np.char.add(args[0], multi_add(args[1:]))
    else:
        return args[0]

def viridis(x):
    colors = colorscale_0.reshape(4,1) + (colorscale_1.reshape(4,1)-colorscale_0.reshape(4,1)) * (x-x.min())/(x.max()-x.min())
    colors=colors.T

    print(colors.shape)
    out = []
    for c in colors:
        out.append(f"rgb({c[0]},{c[1]},{c[2]})")
        #out.append(f"rgb(255,255,0)")
    return np.stack(out)
    
def log_or_0(x):
    ret = x.copy()
    ret[ret>0] = np.log(ret[ret>0])
    ret[ret<=0] = 0
    return ret


In [ ]:
import plotly.io as pio

fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            #opacity=0.4,
            #facecolor=np.random.rand(triangles.shape[0],3)*per_triangle_resolution #per_triangle_resolution[],
            customdata=np.stack([np.arange(len(triangles[:,2])), per_triangle_resolution]).T,
            #customdata=np.stack([np.arange(len(vertices[:,2])), vertices[:,0]]).T,
            # facecolor="blue"
            #facecolor=viridis(log_or_0(per_triangle_resolution))
            hovertemplate="%{x:.2f},%{y:.2f},%{z:.2f}<br>triangle %{customdata[0]}<br>npoints= %{customdata[1]}</extra>",
            #facecolor=viridis(1*(per_triangle_resolution>0))
            #facecolor=viridis(log_or_0(per_triangle_resolution))
            intensity=(per_triangle_resolution),
            intensitymode="cell",
        ),
    ],
    #layout=dict(
    #    scene=dict(aspectmode="data"), scene_camera=dict(eye=dict(x=0.5, y=0, z=0.8))
    #),
)
#fig.update_layout(
#    scene=dict(aspectmode="data"),
#    title=dict(text=f"Normals", x=0.5),
#    autosize=True,
#    height=1000,
#    plot_bgcolor="rgba(0,0,0,0)",
#    paper_bgcolor="rgba(0,0,0,0)",
#    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
#)
#fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig = fig.show()

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode="markers",
            marker=dict(color="blue", size=1),
        ),
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            customdata=np.stack([np.arange(len(triangles[:,2])), per_triangle_resolution]).T,
            # facecolor="blue"
            opacity=1,
            #facecolor=viridis(log_or_0(per_triangle_resolution))
            hovertemplate="%{x:.2f},%{y:.2f},%{z:.2f}<br>tri %{customdata[0]}<br>res %{customdata[1]}</extra>",
            intensity=(per_triangle_resolution>0),
            intensitymode="cell"
        ),
    ],
    layout=dict(
        scene=dict(aspectmode="data"), scene_camera=dict(eye=dict(x=0.5, y=0, z=0.8))
    ),
)
fig.update_layout(
    scene=dict(aspectmode="data"),
    height=1000,
    #plot_bgcolor="rgba(0,0,0,0)",
    #paper_bgcolor="rgba(0,0,0,0)",
    #legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
)

fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig = fig.show()

In [ ]:
vertices[triangles[i]]
vertices[triangles[i]]
vertices[triangles[i]][:,0]

In [ ]:
i = 5665

projected = (transform_to_tri_cc[i] @ points_h.T).T
is_within_triangle = ( 
    (projected[:, 0] >= 0)
    & (projected[:, 1] >= 0)
    & ((projected[:, 1] + projected[:, 0]) <= 1)
    & (
        ((projected[:, 2] > 0) & (projected[:, 2] <= normal_eps_pos))
        | ((projected[:, 2] < 0) & (projected[:, 2] >= normal_eps_neg))
    )
)

is_within_triangle
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode="markers",
            marker=dict(color="blue", size=1),
        ),
        go.Scatter3d(
            x=points[is_within_triangle, 0],
            y=points[is_within_triangle, 1],
            z=points[is_within_triangle, 2],
            mode="markers",
            marker=dict(color="green", size=2),
        ),
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            customdata=np.stack([np.arange(len(triangles[:,2])), per_triangle_resolution]).T,
            # facecolor="blue"
            opacity=0.4,
            #facecolor=viridis(log_or_0(per_triangle_resolution))
            hovertemplate="%{x:.2f},%{y:.2f},%{z:.2f}<br>tri %{customdata[0]}<br>res %{customdata[1]}</extra>",
            intensity=1*(per_triangle_resolution>0),
            intensitymode="cell"
        ),
        go.Mesh3d(
            x=vertices[triangles[i]][:,0],
            y=vertices[triangles[i]][:,1],
            z=vertices[triangles[i]][:,2],
            customdata=np.stack([np.arange(len(triangles[:,2])), per_triangle_resolution]).T,
            color="red",
            opacity=1,
            #facecolor=viridis(log_or_0(per_triangle_resolution))
            hovertemplate="%{x:.2f},%{y:.2f},%{z:.2f}<br>tri %{customdata[0]}<br>res %{customdata[1]}</extra>",
        ),
    ],
    layout=dict(
        scene=dict(aspectmode="data"), scene_camera=dict(eye=dict(x=0.5, y=0, z=0.8))
    ),
)
fig.update_layout(
    scene=dict(aspectmode="data"),
    height=1000,
    #plot_bgcolor="rgba(0,0,0,0)",
    #paper_bgcolor="rgba(0,0,0,0)",
    #legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
)

fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig = fig.show()

In [ ]:
import occlusion

per_triangle_resolution, _ = occlusion.get_incident_points_per_triangle(vertices, triangles, points)

fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            #opacity=0.4,
            #facecolor=np.random.rand(triangles.shape[0],3)*per_triangle_resolution #per_triangle_resolution[],
            customdata=np.stack([np.arange(len(triangles[:,2])), per_triangle_resolution]).T,
            #customdata=np.stack([np.arange(len(vertices[:,2])), vertices[:,0]]).T,
            # facecolor="blue"
            #facecolor=viridis(log_or_0(per_triangle_resolution))
            hovertemplate="%{x:.2f},%{y:.2f},%{z:.2f}<br>triangle %{customdata[0]}<br>npoints= %{customdata[1]}</extra>",
            #facecolor=viridis(1*(per_triangle_resolution>0))
            #facecolor=viridis(log_or_0(per_triangle_resolution))
            intensity=(per_triangle_resolution),
            intensitymode="cell",
        ),
    ],
    #layout=dict(
    #    scene=dict(aspectmode="data"), scene_camera=dict(eye=dict(x=0.5, y=0, z=0.8))
    #),
)
#fig.update_layout(
#    scene=dict(aspectmode="data"),
#    title=dict(text=f"Normals", x=0.5),
#    autosize=True,
#    height=1000,
#    plot_bgcolor="rgba(0,0,0,0)",
#    paper_bgcolor="rgba(0,0,0,0)",
#    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
#)
#fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig = fig.show()